Uses code from a notebook by Jake Vanderplas, edited by A. Mann (information below). 

<small><i>This notebook was put together by [Jake Vanderplas](http://www.vanderplas.com). Source and license info is on [GitHub](https://github.com/jakevdp/sklearn_tutorial/).</i></small>

Written by Donovan Schlekat for ASTR 519.

# Random Forest Aging of Star Clusters

This notebook attempts to age star clusters using supervised machine learning "Random Forest" methods and data from the Milky Way Star Cluster (MWSC) Catalog. 

<small><i>(available at: [https://heasarc.gsfc.nasa.gov/W3Browse/star-catalog/mwsc.html](url) )</i></small>

In [1]:
# Important imports

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [52]:
# Read in the file
# File is the entire MWSC Catalog saved in csv format
csv_file_path = 'MWSC_Catalog.csv'
df = pd.read_csv(csv_file_path)
df_dropped = df.drop([
    'broad_type',
    'cluster_status',
    'log_age_error',
    'metallicity_error'
    ], axis=1)

# Filter data to only contain open clusters with metallicity data
open_metal_df = df_dropped[
    (df['metallicity'].notna()) &
    (df['class'].isin(['OPEN STAR CLUSTER']))
]

# Filter data to only contain globular clusters with metallicity data
glob_metal_df = df_dropped[
    (df['metallicity'].notna()) &
    (df['class'].isin(['GLOBULAR CLUSTER EXTENDED GALACTIC OR EXTRAGALACTIC']))
]

# Filter data to only contain clusters with metallicity data
all_metal_df = df_dropped[(df['metallicity'].notna()) &
    (df['metallicity'].notna())
]

### Cluster Age Classification Using Random Forests

The following cells classify clusters into four categories: ancient (~ $10^9$ yr), old (~ $10^8$ yr), intermediate (~ $10^7$ yr), or young (~ $10^6$ yr).

In [53]:
# Classify data into new categories described above
def categorize_star_cluster_age(log_age):
    if 6 <= log_age < 7:
        return 'Young Star Cluster'
    elif 7 <= log_age < 8:
        return 'Intermediate-Age Star Cluster'
    elif 8 <= log_age < 9:
        return 'Old Star Cluster'
    else:
        return 'Ancient Star Cluster'
    
open_metal_df['age_group'] = open_metal_df['log_age'].apply(categorize_star_cluster_age)
glob_metal_df['age_group'] = glob_metal_df['log_age'].apply(categorize_star_cluster_age)
all_metal_df['age_group'] = all_metal_df['log_age'].apply(categorize_star_cluster_age)

First we'll test the Random Forest Classifier on both sets of data using all of the relevant the paramaters available: galactic location, sky location, radius, number of stars, distance, reddening, and metallicity.

In [37]:
data = open_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

cluster_data_matrix = []
cluster_age_matrix = []

for i in range(len(data)):
    tempList = [lii[i], bii[i], ra[i], dec[i], core_radius[i], num_core_stars[i], distance[i], reddening[i], metallicity[i]]
    cluster_data_matrix.append(tempList)
    cluster_age_matrix.append(age_group[i])

o_cluster_train, o_cluster_test, o_age_train, o_age_test = train_test_split(cluster_data_matrix, cluster_age_matrix, test_size=0.5)

model = RandomForestClassifier(max_depth=5)

model.fit(o_cluster_train, o_age_train)

o_predictions = model.predict(o_cluster_test)

In [38]:
print('Open cluster score: ',metrics.accuracy_score(o_predictions, o_age_test))

Open cluster score:  0.672566371681416


In [28]:
data = glob_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

cluster_data_matrix = []
cluster_age_matrix = []

for i in range(len(data)):
    tempList = [lii[i], bii[i], ra[i], dec[i], core_radius[i], num_core_stars[i], distance[i], reddening[i], metallicity[i]]
    cluster_data_matrix.append(tempList)
    cluster_age_matrix.append(age_group[i])

g_cluster_train, g_cluster_test, g_age_train, g_age_test = train_test_split(cluster_data_matrix, cluster_age_matrix, test_size=0.5)

model = RandomForestClassifier(max_depth=5)

model.fit(g_cluster_train, g_age_train)

g_predictions = model.predict(g_cluster_test)

In [29]:
print('Globular cluster score: ',metrics.accuracy_score(g_predictions, g_age_test))

Globular cluster score:  1.0


Next, we'll try classification based on each parameter individually.

In [31]:
data = open_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

param_list = [lii, bii, ra, dec, core_radius, num_core_stars, distance, reddening, metallicity]
cluster_age_matrix = []
param_success_matrix = []
param_success_matrix.append(['Cluster type: ','Open'])

for i in range(len(data)):
    cluster_age_matrix.append(age_group[i])

for param in param_list:
    param_name = param.name
    param = param.to_numpy()
    param = param.reshape(-1, 1)
    o_cluster_train, o_cluster_test, o_age_train, o_age_test = train_test_split(param, cluster_age_matrix, test_size=0.5)

    model = RandomForestClassifier(max_depth=5)

    model.fit(o_cluster_train, o_age_train)

    o_predictions = model.predict(o_cluster_test)
    accuracy = metrics.accuracy_score(o_predictions, o_age_test)

    param_success_matrix.append([param_name, accuracy])

param_success_matrix

[['Cluster type: ', 'Open'],
 ['lii', 0.4247787610619469],
 ['bii', 0.5221238938053098],
 ['ra', 0.49557522123893805],
 ['dec', 0.37168141592920356],
 ['core_radius', 0.4778761061946903],
 ['num_core_stars', 0.39823008849557523],
 ['distance', 0.6548672566371682],
 ['e_bv', 0.415929203539823],
 ['metallicity', 0.45132743362831856]]

In [32]:
data = glob_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

param_list = [lii, bii, ra, dec, core_radius, num_core_stars, distance, reddening, metallicity]
cluster_age_matrix = []
param_success_matrix = []
param_success_matrix.append(['Cluster type: ','Globular'])

for i in range(len(data)):
    cluster_age_matrix.append(age_group[i])

for param in param_list:
    param_name = param.name
    param = param.to_numpy()
    param = param.reshape(-1, 1)
    g_cluster_train, g_cluster_test, g_age_train, g_age_test = train_test_split(param, cluster_age_matrix, test_size=0.5)

    model = RandomForestClassifier(max_depth=5)

    model.fit(g_cluster_train, g_age_train)

    o_predictions = model.predict(g_cluster_test)
    accuracy = metrics.accuracy_score(g_predictions, g_age_test)

    param_success_matrix.append([param_name, accuracy])

param_success_matrix

[['Cluster type: ', 'Globular'],
 ['lii', 1.0],
 ['bii', 1.0],
 ['ra', 1.0],
 ['dec', 1.0],
 ['core_radius', 1.0],
 ['num_core_stars', 1.0],
 ['distance', 1.0],
 ['e_bv', 1.0],
 ['metallicity', 1.0]]

A couple of things to note: based on our extremely arbitrarily defined categories, <i>all globular clusters are defined as ancient</i>. This is why the success score is always perfect: the algorithm recognizes that every data point, which are all globular clusters, belongs to this particular category, as there are no other categories to compete.

Let's try something new: going back to the open cluster classifier, running the random forests algorithm mulitple times, and picking the best one as our "optimal" model.

In [51]:
data = open_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

cluster_data_matrix = []
cluster_age_matrix = []

for i in range(len(data)):
    tempList = [lii[i], bii[i], ra[i], dec[i], core_radius[i], num_core_stars[i], distance[i], reddening[i], metallicity[i]]
    cluster_data_matrix.append(tempList)
    cluster_age_matrix.append(age_group[i])

accuracy = []

for i in range(100):
    o_cluster_train, o_cluster_test, o_age_train, o_age_test = train_test_split(cluster_data_matrix, cluster_age_matrix, test_size=0.5)

    model = RandomForestClassifier(max_depth=5)

    model.fit(o_cluster_train, o_age_train)

    o_predictions = model.predict(o_cluster_test)
    accuracy.append(metrics.accuracy_score(o_predictions, o_age_test))

best_model = np.max(accuracy)
print(best_model)

0.6991150442477876


Finally, we will test the random forest on all clusters with metallicity data, globular and open.

In [59]:
data = all_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

cluster_data_matrix = []
cluster_age_matrix = []

for i in range(len(data)):
    tempList = [lii[i], bii[i], ra[i], dec[i], core_radius[i], num_core_stars[i], distance[i], reddening[i], metallicity[i]]
    cluster_data_matrix.append(tempList)
    cluster_age_matrix.append(age_group[i])

accuracy = []

for i in range(100):
    a_cluster_train, a_cluster_test, a_age_train, a_age_test = train_test_split(cluster_data_matrix, cluster_age_matrix, test_size=0.5)

    model = RandomForestClassifier(max_depth=5)

    model.fit(a_cluster_train, a_age_train)

    a_predictions = model.predict(a_cluster_test)
    accuracy.append(metrics.accuracy_score(a_predictions, a_age_test))

best_model = np.max(accuracy)
print(best_model)

0.8298969072164949


And now the same data set, but testing each parameter individually:

In [60]:
data = all_metal_df
data = data.reset_index(drop=True)
headers = data.columns

lii = data['lii']
bii = data['bii']
ra = data['ra']
dec = data['dec']
core_radius = data['core_radius']
num_core_stars = data['num_core_stars']
distance = data['distance']
reddening = data['e_bv']
age_group = data['age_group']
metallicity = data['metallicity']

param_list = [lii, bii, ra, dec, core_radius, num_core_stars, distance, reddening, metallicity]
cluster_age_matrix = []
param_success_matrix = []
param_success_matrix.append(['Cluster type: ','All'])

for i in range(len(data)):
    cluster_age_matrix.append(age_group[i])

for param in param_list:
    param_name = param.name
    param = param.to_numpy()
    param = param.reshape(-1, 1)
    a_cluster_train, a_cluster_test, a_age_train, a_age_test = train_test_split(param, cluster_age_matrix, test_size=0.5)

    model = RandomForestClassifier(max_depth=5)

    model.fit(a_cluster_train, a_age_train)

    o_predictions = model.predict(a_cluster_test)
    accuracy = metrics.accuracy_score(a_predictions, a_age_test)

    param_success_matrix.append([param_name, accuracy])

param_success_matrix

[['Cluster type: ', 'All'],
 ['lii', 0.4948453608247423],
 ['bii', 0.4484536082474227],
 ['ra', 0.42783505154639173],
 ['dec', 0.4742268041237113],
 ['core_radius', 0.3865979381443299],
 ['num_core_stars', 0.5257731958762887],
 ['distance', 0.4690721649484536],
 ['e_bv', 0.4536082474226804],
 ['metallicity', 0.4690721649484536]]